In [1]:
import os
import numpy as np
import pandas as pd

from IPython.display import display

In [2]:
# PATHS
DATA_ROOT = "generalized_contrastive_loss"

MSLS_MODELS = os.path.join(DATA_ROOT, "Models", "MSLS")
RESULTS_ROOT = os.path.join(DATA_ROOT, "results", "MSLS", "val")
PREDICTION_PATH = os.path.join(RESULTS_ROOT, "MSLS_resnext_GeM_480_GCL_predictions.txt.npy")
FULL_PREDICTION_PATH = os.path.join(RESULTS_ROOT, "MSLS_resnext_GeM_480_GCL_predictions.txt")
RESULTS_NPY_TEXT = "MSLS_resnext_GeM_480_GCL_{city}_{file_type}.npy"

DATASET_ROOT = os.path.join(DATA_ROOT, "msls")
DATASET_TEST = os.path.join(DATASET_ROOT, "test")
DATASET_VAL = os.path.join(DATASET_ROOT, "train_val")

In [3]:
pred_data = np.load(PREDICTION_PATH)
print(f"Prediction NPY: {pred_data.shape}")

Prediction NPY: (4525, 31)


In [4]:
map_feature_data_cph = np.load(os.path.join(RESULTS_ROOT, RESULTS_NPY_TEXT.format(city="cph", file_type="mapfeats")))
print(f"Map Feature CPH: {map_feature_data_cph.shape}")
query_feature_data_cph = np.load(os.path.join(RESULTS_ROOT, RESULTS_NPY_TEXT.format(city="cph", file_type="queryfeats")))
print(f"Query Feature CPH: {query_feature_data_cph.shape}")
distances_cph = np.load(os.path.join(RESULTS_ROOT, RESULTS_NPY_TEXT.format(city="cph", file_type="distances")))
print(f"Distances CPH: {distances_cph.shape}")

Map Feature CPH: (12601, 2048)
Query Feature CPH: (6595, 2048)
Distances CPH: (6595, 12601)


In [5]:
map_feature_data_sf = np.load(os.path.join(RESULTS_ROOT, RESULTS_NPY_TEXT.format(city="sf", file_type="mapfeats")))
print(f"Map Feature SF: {map_feature_data_sf.shape}")
query_feature_data_sf = np.load(os.path.join(RESULTS_ROOT, RESULTS_NPY_TEXT.format(city="sf", file_type="queryfeats")))
print(f"Query Feature SF: {query_feature_data_sf.shape}")
distances_sf = np.load(os.path.join(RESULTS_ROOT, RESULTS_NPY_TEXT.format(city="sf", file_type="distances")))
print(f"Distances SF: {distances_sf.shape}")

Map Feature SF: (6315, 2048)
Query Feature SF: (4525, 2048)
Distances SF: (4525, 6315)


In [6]:
# read the predictions text file
df = pd.read_fwf(FULL_PREDICTION_PATH, header=None)
df['combined'] = df.drop(0, axis=1).values.tolist()
df = df[[0, 'combined']]
df.columns = ['query_id', 'retrieved_ids']
display(df)

,query_id,retrieved_ids
0,x3vA7Bk0HNI6rGkDpDZQUQ,"[X9V1oGRaAEFjq5jufrklTQ, E7gcrCyitkguCnMzoEwm0..."
1,U9Vj0IV4q1psciXpj51F_w,"[X9V1oGRaAEFjq5jufrklTQ, HU9GEfLAB9pm5RmjW4MLh..."
2,Eh1NwQjH4jbKcWqVJ4ZsJg,"[X9V1oGRaAEFjq5jufrklTQ, _Eq8EgtwLGiMFc7VJdb-Y..."
3,1RKCGBAWsZbi5dj3vR2mlw,"[_Eq8EgtwLGiMFc7VJdb-YQ, X9V1oGRaAEFjq5jufrklT..."
4,LdiYwYkqgUfc1IYDu5ov9A,"[Z4MR4AHQufgsCwiBiqQ23A, _Eq8EgtwLGiMFc7VJdb-Y..."
...,...,...
11115,BdY6m-mEqcF_EAgWXhNhMQ,"[c4u-9p_hYbMqZMV9M9NJOw, HWs14OYXLRbvsVJ7sHJX0..."
11116,MISH40wohJPWQg2AQL91ww,"[c4u-9p_hYbMqZMV9M9NJOw, L8wuXgwUX0RV_CZ9cDni7..."
11117,8YnuOWWGLvmT1S405w3ydw,"[L8wuXgwUX0RV_CZ9cDni7A, oaavK1hzB6S7lK3d4zNpU..."
11118,yqa5OwamR2zj8KomdqC3FQ,"[qOFK3KN5VcWMzBde01gPiA, S-MaMTx8Gq1AW144nSerd..."
